<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/rule_ml_pair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Импорт**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [2]:
from IPython.display import SVG, display, HTML

In [3]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Drive already mounted at ./content; to attempt to forcibly remount, call drive.mount("./content", force_remount=True).


In [4]:
import spacy
import numpy as np

import gensim
from gensim.models import Word2Vec 
from gensim.models import FastText

from nltk.tokenize import RegexpTokenizer
from nltk import  sent_tokenize
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
from graphviz import Source
from sklearn import tree, svm, linear_model
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [6]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

annotations.csv		neuroleptic_antidepr2.txt
annotation_table.csv	neuroleptic_antidepr2.txt.1
annotation_table.csv.1	neuroleptic_antidepr2.txt.2
annotation_table.csv.2	neuroleptic_antidepr2.txt.3
annotation_table.csv.3	neuroleptic_antidepr2.txt.4
annotation_table.csv.4	neuroleptic_antidepr2.txt.5
annotation_table.csv.5	neuroleptic_antidepr2.txt.6
annotation_table.csv.6	neuroleptic_antidepr2.txt.7
annotation_table.csv.7	neuroleptic_antidepr2.txt.8
annotation_table.csv.8	rakpobedim_ru2.txt
content			rakpobedim_ru2.txt.1
hv-info_gepatit2.txt	rakpobedim_ru2.txt.2
hv-info_gepatit2.txt.1	rakpobedim_ru2.txt.3
hv-info_gepatit2.txt.2	rakpobedim_ru2.txt.4
hv-info_gepatit2.txt.3	rakpobedim_ru2.txt.5
hv-info_gepatit2.txt.4	rakpobedim_ru2.txt.6
hv-info_gepatit2.txt.5	rakpobedim_ru2.txt.7
hv-info_gepatit2.txt.6	rakpobedim_ru2.txt.8
hv-info_gepatit2.txt.7	sample_data
hv-info_gepatit2.txt.8
.file-revisions-by-id
.shortcut-targets-by-id
MyDrive
.Trash


In [7]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

## **Обработка данных**

In [8]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit2.txt

--2021-05-24 23:35:42--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11655409 (11M) [text/plain]
Saving to: ‘neuroleptic_antidepr2.txt.9’

neuroleptic_antidep 100%[===================>]  11.12M  30.0MB/s    in 0.4s    

2021-05-24 23:35:43 (30.0 MB/s) - ‘neuroleptic_antidepr2.txt.9’ saved [11655409/11655409]

--2021-05-24 23:35:43--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:

In [9]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv

--2021-05-24 23:35:46--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-05-24 23:35:46 ERROR 404: Not Found.

--2021-05-24 23:35:46--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2100361 (2.0M) [text/plain]
Saving to: ‘annotation_table.csv.9’

annotation_table.cs 100%[===================>]   2.00M  --.-KB/s    in 0.

In [10]:
handler = open('neuroleptic_antidepr2.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()

handler = open('hv-info_gepatit2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

handler = open('rakpobedim_ru2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [11]:
posts = my_txt.split('----')

## **Обучение векторизатора**

In [12]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]


sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]
ft = FastText(sentences_tokenized, window=5, min_count=5, workers=4,sg=1)

Обработка битых символов, чтобы pandas мог открыть таблицу.

In [13]:
count = 0
with open('annotation_table.csv') as csvfile:
  with open('annotations.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    for new_row in csvfile:
      count +=1
      # print(i.split('\t'))
      if len(new_row.split('\t')) > 19:
        new_row = new_row.split('\t')[:20]
        new_row[-1] = new_row[-1][:-1] # cut \n
      writer.writerow(new_row)
count

27019

In [14]:
# # with open('coref_df_anns60k.csv') as csvfile:
# with open('new_anns60k.csv') as csvfile:
#   reader = csv.reader(csvfile, delimiter='\t')
#   with open('new_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     c = 0
#     for row in reader:
#       c +=1
#       new_row = [i if i and not re.match(r'\t+?', i) else 0 for i in row]
#       if len(new_row) > 9:
#         new_row = new_row[:9]
#       else:
#         new_row = [0 for i in range(9)]
#       writer.writerow(new_row)
# c


In [15]:
# !rm new_anns60k.csv.1 new_anns60k.csv.2 new_df_anns60k.csv new_anns60k.csv
!ls
# !rm annotation_table.csv.2 annotation_table.csv.1 annotation_table.csv annotations.csv

annotations.csv		neuroleptic_antidepr2.txt
annotation_table.csv	neuroleptic_antidepr2.txt.1
annotation_table.csv.1	neuroleptic_antidepr2.txt.2
annotation_table.csv.2	neuroleptic_antidepr2.txt.3
annotation_table.csv.3	neuroleptic_antidepr2.txt.4
annotation_table.csv.4	neuroleptic_antidepr2.txt.5
annotation_table.csv.5	neuroleptic_antidepr2.txt.6
annotation_table.csv.6	neuroleptic_antidepr2.txt.7
annotation_table.csv.7	neuroleptic_antidepr2.txt.8
annotation_table.csv.8	neuroleptic_antidepr2.txt.9
annotation_table.csv.9	rakpobedim_ru2.txt
content			rakpobedim_ru2.txt.1
hv-info_gepatit2.txt	rakpobedim_ru2.txt.2
hv-info_gepatit2.txt.1	rakpobedim_ru2.txt.3
hv-info_gepatit2.txt.2	rakpobedim_ru2.txt.4
hv-info_gepatit2.txt.3	rakpobedim_ru2.txt.5
hv-info_gepatit2.txt.4	rakpobedim_ru2.txt.6
hv-info_gepatit2.txt.5	rakpobedim_ru2.txt.7
hv-info_gepatit2.txt.6	rakpobedim_ru2.txt.8
hv-info_gepatit2.txt.7	rakpobedim_ru2.txt.9
hv-info_gepatit2.txt.8	sample_data
hv-info_gepatit2.txt.9


In [16]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('annotations.csv', delimiter='\t')
anns_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,sing
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27013,и,0,22,68.0,0.0,2385.0,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,0,0.0,NaN
27014,таможенные,0,22,68.0,0.0,2386.0,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,0,0.0,plur
27015,пошлины,0,22,68.0,0.0,2387.0,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,0,0.0,plur
27016,как,0,22,68.0,0.0,2388.0,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,0,0.0,NaN


In [17]:
tok_list = []
for token in anns_df.TOKEN:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['TOKEN_VECT'] = tok_list


tok_list = []
for token in anns_df.HEAD:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['HEAD_VECT'] = tok_list

In [18]:
# tok_list = []
# for sent in 
# for token in anns_df.HEAD:
#   try:
#     tok_list.append(ft.wv.__getitem__(token.lower()))
#   except KeyError:
#     tok_list.append(0)
#     print(token, 'here')
# anns_df['TOKEN_VECT'] = tok_list

In [19]:
anns_df = anns_df.loc[(anns_df['TOPIC_NUM'].isin(list(range(30)))) | (anns_df['SENT_NUM'].isin(list(range(10000))))]
anns_df = anns_df.reset_index().drop(['index'], axis=1)

In [20]:
topics = list(set(anns_df.TOPIC_NUM.values))
topics
sent_vect_list = []
# номера предложений топика
for topic_num in topics:

  sents_in_topic = list(set(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num].SENT_NUM))
  topic_len = len(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num])
  for sent_num in sents_in_topic:
    tok_vects = anns_df.loc[(anns_df['TOPIC_NUM'] == topic_num) & (anns_df['SENT_NUM'] == sent_num)].TOKEN_VECT
    try:
      sent_vect = sum(list(tok_vects))/len(tok_vects)
    except ZeroDivisionError:
      sent_vect = 0
      print(anns_df.loc[(anns_df['SENT_NUM'] == sent_num)])
    for i in range(len(tok_vects)):
      sent_vect_list.append(sent_vect)
len(sent_vect_list)
# если не класть нулевых предложений - будет длина датафрейма, как он раньше импортировлся, но поздно.

26497

In [21]:
anns_df['SENT_VECT'] = sent_vect_list

Проверим данные

In [22]:
anns_df.SENT_NUM[18]

1.0

In [23]:
# topics = list(set(df.TOPIC_NUM.values)) 
# for topic_num in topics:
#   # if topic_num == 2:
#   #   break
#   # извлекаем предложения топика
#   sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
#   for sent_num in sents_in_topic:
#     sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num)]

In [24]:
# vvv = 0
# with open('content/MyDrive/coref_fasttext60k_1.csv') as csvfile:
#   with open('new_coref_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     for new_row in csvfile:
#       vvv +=1
#       # print(i.split('\t'))
#       if len(new_row.split('\t')) > 9:
#         new_row = new_row.split('\t')[:9]
#       writer.writerow(new_row)
# vvv

In [25]:
# cor_df = pd.read_csv('content/MyDrive/coref_fasttext60k_1.csv', delimiter=' ')
# cor_df

In [26]:
# cosine_similarity([np.matrix(cor_df.TOKEN_VECT[2]).A[0]], [np.matrix(cor_df.TOKEN_VECT[3]).A[0]])[-1][-1]

# **VECTORS HANDLING**

Изменяем значения векторов предложений, главного слова группы, зависимых слов группы, и токена на косинусную дистанцию с семантической осью.

Сначала, изменим строковое представление на массив.

In [27]:
# cor_df.TOKEN_VECT = [np.matrix(i).A[0] for i in cor_df.TOKEN_VECT]
# cor_df.HEAD_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_VECT]
# cor_df.SENT_VECT = [np.matrix(i).A[0] for i in cor_df.SENT_VECT]
# cor_df.HEAD_CHILDS_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_CHILDS_VECT]


Добавим размеченные данные, и посмотрим, что получилось.

In [28]:
# cor_df['NER'] = anns_df['NER']
# cor_df['COREFERENCE_CLUSTER'] = anns_df['COREFERENCE_CLUSTER']
# cor_df['ANN_TOKEN'] = anns_df['TOKEN']
# cor_df

In [29]:
# cor_df.loc()

Ограничим датафрейм теми данными, которые были размечены, чтобы была возможность провести оценку системы.

In [30]:
df = anns_df.iloc[:15000,:]

Всего наименований препаратов размечено (это не обязательно антецеденты, любое название препарата).

In [31]:
len(df.loc[df['NER'] == 1])

581

Из них местоимений (прономинальные анафоры).

In [32]:
len(df.loc[df['NER'] == 3])

113

И номинативные анафоры существительные, такие как "препарат" и "лекарство". В основном это они и есть.

In [33]:
len(df.loc[df['NER'] == 2])

215

In [34]:
len(df.loc[df['NER'].isin([1,2,3])])

909

In [35]:
# df['NER'] = [int(i) for i in df['NER']]
# df['COREFERENCE_CLUSTER'] = [int(i) for i in df['COREFERENCE_CLUSTER']]

Посмотрим на данные.

In [36]:
df.loc[df['NER'].isin([1,2,3])]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.52989316, 0.66116303, 0.80405414, 0.1667848...","[-0.05602419, 1.3713305, 0.48728576, 1.3173157...","[0.15188894, 0.47630128, 0.19712001, -0.135239..."
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,"[0.5861964, 0.9571688, 0.59529054, 0.15614, 0....","[0.52989316, 0.66116303, 0.80405414, 0.1667848...","[0.15188894, 0.47630128, 0.19712001, -0.135239..."
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,"[0.13672307, 0.77963775, 0.636794, 0.1634183, ...","[0.5861964, 0.9571688, 0.59529054, 0.15614, 0....","[0.15188894, 0.47630128, 0.19712001, -0.135239..."
21,лекарством,0,1,1.0,1.0,21.0,inan,ablt,neut,NaN,NOUN,conj,нет,NaN,NaN,NaN,PRED,2,0.0,sing,"[-0.310879, 1.2153386, 0.30751365, 0.11475402,...","[0.8418265, 0.31743556, 0.3601378, 0.020493388...","[0.105063275, 0.4141047, 0.4832334, -0.0696449..."
142,рисполепта,0,1,1.0,13.0,142.0,inan,nomn,femn,NaN,NOUN,nmod,дозировку,inan,accs,femn,NOUN,1,142.0,sing,"[0.56313324, 0.8396723, 0.67768383, 0.06166613...","[0.61126375, 1.1168321, 0.42246604, 0.38041788...","[0.08132484, 0.7005222, 0.35762572, -0.2417465..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14807,абраксан,1,15,18.0,3.0,2215.0,anim,gent,femn,NaN,NOUN,obj,спросите,NaN,NaN,NaN,VERB,1,2215.0,plur,"[0.54627293, 0.71837795, 0.54261947, -0.002646...","[0.3498348, 0.0620882, -0.11074386, 0.1959932,...","[0.3786574, 0.35384712, 0.16447143, 0.07777113..."
14809,гемцитабином,1,15,18.0,3.0,2217.0,inan,ablt,masc,NaN,NOUN,nmod,абраксан,anim,gent,femn,NOUN,1,2217.0,sing,"[0.016098425, 0.83841366, 0.21292609, 0.153194...","[0.54627293, 0.71837795, 0.54261947, -0.002646...","[0.3786574, 0.35384712, 0.16447143, 0.07777113..."
14856,фолфиринокс,1,15,18.0,8.0,2264.0,inan,nomn,masc,NaN,NOUN,nmod,схема,inan,nomn,femn,NOUN,1,2264.0,sing,"[0.45440283, 0.43411574, 0.26088437, -0.195037...","[0.49486047, -0.14809924, 0.91566277, -0.54630...","[0.3786574, 0.35384712, 0.16447143, 0.07777113..."
14887,иринотекану,1,15,19.0,0.0,2295.0,inan,datv,masc,NaN,NOUN,nmod,аннотации,inan,nomn,femn,NOUN,1,2295.0,sing,"[0.61597925, 0.71795684, 0.18081982, 0.0862167...","[0.001423307, 0.9877145, -0.17775936, 0.075810...","[0.3786574, 0.35384712, 0.16447143, 0.07777113..."


# **SEMANTIC AXIS**

Реализация метода семантических осей.
Сначала, выбираем первые предложения каждого топика. Посмотрим на них.

In [37]:
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.52989316, 0.66116303, 0.80405414, 0.1667848...","[-0.05602419, 1.3713305, 0.48728576, 1.3173157...","[0.15188894, 0.47630128, 0.19712001, -0.135239..."
576,зелдокс,0,2,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.4476504, 0.92441326, 0.5744501, -0.11262302...","[-0.05602419, 1.3713305, 0.48728576, 1.3173157...","[0.32248265, 0.48711208, 0.23260376, -0.003890..."
1870,оланзапин,0,3,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.39558592, 0.6582631, 0.578083, -0.03835348,...","[-0.05602419, 1.3713305, 0.48728576, 1.3173157...","[0.42017376, 0.35683358, 0.1135206, 0.00777316..."
5555,галоперидол,0,4,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,1,2.0,sing,"[0.28657344, 1.2659293, 0.263747, 0.049427845,...","[-0.60645026, 0.40584835, -0.28015998, -0.1943...","[0.22129436, 0.19725913, 0.2205431, 0.00559320..."
6200,тиаприд,0,5,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,3.0,sing,"[0.50418127, 0.42649567, 0.6586662, -0.1669650...","[-0.05602419, 1.3713305, 0.48728576, 1.3173157...","[0.21120502, 0.2751641, 0.23118126, 0.00680639..."
8450,сероквель,0,6,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.7197289, 0.8970752, 0.6137448, 0.08047691, ...","[-0.05602419, 1.3713305, 0.48728576, 1.3173157...","[0.17428371, 0.39893225, 0.13767146, 0.0223892..."
8541,день,0,7,0.0,0.0,2.0,inan,accs,masc,NaN,NOUN,ROOT,день,inan,accs,masc,NOUN,0,0.0,sing,"[-0.68165106, 0.960153, -0.12893267, 0.4452025...","[-0.68165106, 0.960153, -0.12893267, 0.4452025...","[0.15323366, 0.31769386, 0.2856932, -0.0074439..."
11117,500,0,8,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,nummod,мг,inan,nomn,masc,NOUN,0,0.0,NaN,"[0.62875265, 0.568246, 1.1540365, 0.36850327, ...","[0.17638081, 0.25628808, 0.92794704, 0.0974672...","[0.22877258, 0.36473462, 0.2783909, 0.02115054..."
11787,капецитабин,0,10,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,ROOT,капецитабин,anim,nomn,masc,NOUN,1,2.0,sing,"[0.4853327, 0.9936661, 0.47487772, -0.21124959...","[0.4853327, 0.9936661, 0.47487772, -0.21124959...","[0.26554897, 0.39730182, 0.23929222, -0.000707..."
11806,иресса,0,11,0.0,0.0,2.0,NaN,NaN,NaN,NaN,GRND,nmod,отдам,NaN,NaN,NaN,VERB,1,2.0,NaN,"[-0.063725516, 0.6332471, 0.67203027, -0.40263...","[0.51499057, 0.52115315, 0.39133263, -0.690674...","[0.26554897, 0.39730182, 0.23929222, -0.000707..."


In [38]:
# sum(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)/len(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)

Суммируем верктора токенов, главных слов, предложения целиком, и зависимых слов. Нормализуем.

In [39]:

token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
# head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [40]:
df.TOKEN_VECT[710]

0

Функция, которая заменяет столбец с векторами на столбец со значениями косинусной дистанции с соответствующей семантической осью.


In [41]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col



In [42]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
# make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [43]:
df.TOKEN[1]

'препарат'

In [44]:
# df['SUM_VECT']= [(df.TOKEN_VECT[i] + df.SENT_VECT[i])/3 for i in range(len(df.SENT_VECT))]

Посмотрим на обработанные данные.

In [45]:
df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur,0.478800,0.513731,0.914983
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,sing,0.534916,0.513731,0.914983
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,0.799821,0.881828,0.914983
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,0.813941,0.574603,0.914983
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,0.730241,0.680588,0.914983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,эта,0,16,3.0,0.0,61.0,NaN,nomn,femn,NaN,ADJF,det,вакцина,inan,nomn,femn,NOUN,0,0.0,sing,0.397897,0.456912,0.981547
14996,вакцина,0,16,3.0,0.0,62.0,inan,nomn,femn,NaN,NOUN,nsubj,аукнется,NaN,NaN,NaN,VERB,0,0.0,sing,0.506270,0.574207,0.981547
14997,аукнется,0,16,3.0,0.0,63.0,NaN,NaN,NaN,3per,VERB,csubj,неизвестно,NaN,NaN,NaN,ADVB,0,0.0,sing,0.511537,0.545054,0.981547
14998,на,0,16,3.0,0.0,64.0,NaN,NaN,NaN,NaN,PREP,case,организме,inan,loct,masc,NOUN,0,0.0,NaN,0.444679,0.488556,0.981547


# **ПРАВИЛОВЫЙ АЛГОРИТМ**

In [46]:
# df.SENT_NUM = [int(i) for i in df['SENT_NUM']]
# df.POST_NUM = [int(i) for i in df['POST_NUM']]
# # df.TOKEN_NUM = [float(i) for i in df['TOKEN_NUM']]
df.TOPIC_NUM = [int(i) for i in df['TOPIC_NUM']]
df.IS_ANSWER = [int(i) for i in df['TOPIC_NUM']]
# df.NER = [int(i) for i in df['NER']]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [47]:
topics = list(set(df.TOPIC_NUM.values))
topics

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [48]:
df.loc[df['TOKEN_NUM'] == 344 ]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
344,рисполепта,1,1,2.0,4.0,344.0,inan,nomn,femn,NaN,NOUN,nmod,приеме,inan,loct,masc,NOUN,1,344.0,sing,0.744327,0.474823,0.943423
918,вышло,2,2,6.0,3.0,344.0,NaN,NaN,neut,NaN,VERB,ROOT,вышло,NaN,NaN,neut,VERB,0,0.0,sing,0.500822,0.473973,0.959016
2212,в,3,3,5.0,8.0,344.0,NaN,NaN,NaN,NaN,PREP,case,использовании,inan,loct,neut,NOUN,0,0.0,NaN,0.452540,0.508909,0.959896
5897,и,4,4,6.0,7.0,344.0,NaN,NaN,NaN,NaN,CONJ,cc,выписывались,NaN,NaN,NaN,VERB,0,0.0,NaN,0.416583,0.441735,0.989138
6542,в,5,5,17.0,0.0,344.0,NaN,NaN,NaN,NaN,PREP,case,мозгу,inan,loc2,masc,NOUN,0,0.0,NaN,0.452540,0.392281,0.992058
8883,их,7,7,0.0,32.0,344.0,NaN,accs,NaN,3per,NPRO,obj,принимать,NaN,NaN,NaN,INFN,0,0.0,plur,0.393018,0.493345,0.957022
11513,отвратно,9,9,0.0,39.0,344.0,NaN,NaN,NaN,NaN,ADVB,advmod,перенесла,NaN,NaN,femn,VERB,0,0.0,NaN,0.487285,0.502938,0.979763
12936,будет,15,15,0.0,31.0,344.0,NaN,NaN,NaN,3per,VERB,advcl,проходить,NaN,NaN,NaN,INFN,0,0.0,sing,0.443375,0.581290,0.973762


In [49]:

topic_num = 1
topic_df = df.loc[df['TOPIC_NUM'] == topic_num]
# номера предложений топика
sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
sents_in_topic
# достать пост
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1)]
# достать конкретное предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)]
# достать токен из предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)]


,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
5,не,1,1,1.0,0.0,5.0,NaN,NaN,NaN,NaN,PRCL,advmod,знаю,NaN,NaN,NaN,VERB,0,0.0,NaN,0.425436,0.459552,0.914983
6,знаю,1,1,1.0,0.0,6.0,NaN,NaN,NaN,1per,VERB,ROOT,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.509180,0.459552,0.914983
7,стоило,1,1,1.0,0.0,7.0,NaN,NaN,neut,NaN,VERB,ccomp,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.488086,0.459552,0.914983
8,ли,1,1,1.0,0.0,8.0,NaN,NaN,NaN,NaN,CONJ,advmod,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.492087,0.452047,0.914983
9,мне,1,1,1.0,0.0,9.0,NaN,datv,NaN,1per,NPRO,iobj,стоило,NaN,NaN,neut,VERB,0,0.0,sing,0.517417,0.452047,0.914983
10,писать,1,1,1.0,0.0,10.0,NaN,NaN,NaN,NaN,INFN,csubj,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.418900,0.452047,0.914983


In [50]:
for i in df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)].values:
  print(i[-5])

0.0
0.0
0.0
0.0
0.0
0.0


In [51]:
# анафоры
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)
& (df['NER'] > 1)].values[0]

array(['лекарством', 1, 1, 1.0, 1.0, 21.0, 'inan', 'ablt', 'neut', nan,
       'NOUN', 'conj', 'нет', nan, nan, nan, 'PRED', 2, 0.0, 'sing',
       0.4469050467014313, 0.5528465509414673, 0.8430876135826111],
      dtype=object)

In [52]:
# sents_in_topic 
# topic_num
# df = df.reset_index()

In [53]:
# knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
# knn_clf.score(X_test, y_test)

In [54]:
def rule_eval(my_X, my_y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(my_X)):
    if my_X[i] != 0:
      if my_X[i] == my_y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif my_X[i] == 0:
      if my_X[i] == my_y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')


In [55]:
topics = list(set(df.TOPIC_NUM.values))
topics

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [56]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
anf_len = 0
ant_len = 0
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  # if post_num == 3:
  #   break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))


  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))
    # if post_num == 3:
    #   break

    for sent_num in sents_in_post:

      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      current_anaphors = sent.loc[(sent['NER'] > 1)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]
      anf_len += len(current_anaphors)
      ant_len += len(current_antecedents)
      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) >= 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            # в противном случае присваиваем нулевой класс
            ann_clusters.append(anfr[-5])
            # и класс, который определило правило(антецедента из предложения)
            my_clusters.append(0.0)

      # если больше одного!!
          
      # если в предложении его нет =(
      elif len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]
          # проверяем, а антецеденты-то есть?
          if not current_antecedents.TOKEN.values.any():
              # присваиваем нулевой класс, если нет даже в топике
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              # для оценки
              my_clusters.append(0.0)
              continue
          # ну и если они есть
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            # try:
            if anfr[-4] == current_antecedents.NUMBER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
              # print(ann_clusters, my_clusters, 'am I here?')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[-1]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      # print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      # print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
print(my_clusters)
print('---')
print(ann_clusters)
print('---')
print(len(my_clusters), len(ann_clusters))
print(anf_len, ant_len)




[2.0, 0.0, 0.0, 344.0, 2.0, 2.0, 440.0, 2.0, 2.0, 540.0, 2.0, 11.0, 11.0, 46.0, 108.0, 108.0, 182.0, 182.0, 182.0, 182.0, 375.0, 434.0, 461.0, 658.0, 658.0, 693.0, 755.0, 755.0, 829.0, 829.0, 829.0, 829.0, 1022.0, 1081.0, 1081.0, 1142.0, 26.0, 0.0, 0.0, 0.0, 183.0, 0.0, 0.0, 0.0, 2.0, 2.0, 575.0, 679.0, 0.0, 0.0, 830.0, 0.0, 0.0, 987.0, 0.0, 1379.0, 0.0, 1483.0, 0.0, 0.0, 0.0, 1750.0, 1750.0, 0.0, 1844.0, 0.0, 2031.0, 2053.0, 0.0, 2058.0, 2155.0, 0.0, 2156.0, 0.0, 2344.0, 2.0, 0.0, 2711.0, 2711.0, 2766.0, 2766.0, 2766.0, 0.0, 0.0, 2.0, 0.0, 3125.0, 3125.0, 3270.0, 3350.0, 3350.0, 0.0, 0.0, 0.0, 3619.0, 0.0, 2.0, 5.0, 5.0, 2.0, 114.0, 0.0, 174.0, 0.0, 0.0, 0.0, 0.0, 287.0, 0.0, 287.0, 287.0, 365.0, 365.0, 0.0, 0.0, 0.0, 449.0, 0.0, 449.0, 449.0, 527.0, 527.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 10.0, 27.0, 41.0, 41.0, 41.0, 2.0, 2.0, 135.0, 135.0, 0.0, 218.0, 231.0, 239.0, 239.0, 256.0, 270.0, 270.0, 270.0, 2.0, 2.0, 2.0, 447.0, 482.0, 482.0, 521.0, 521.0, 521.0, 521.0, 0.0, 0.0, 0.0, 0.0, 0.

In [57]:
rule_eval(my_clusters, ann_clusters)

111  true positive
74  false positive
75  true negative
68  false negative
---------
0.6  presicion
0.6201117318435754  recall
0.6098901098901098  f-score


# **МОДЕЛИ.**

Факторизуем троковые данные, чтобы вместо строковых представлений классов были численные.

In [58]:
for_factorize = 'ANIMACY 	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS NUMBER'.split()
for_factorize

['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS',
 'NUMBER']

In [59]:
_, lbl = pd.factorize(df.ANIMACY)
print(lbl, )
_

Index(['inan', 'anim'], dtype='object')


array([-1,  0,  1, ..., -1, -1,  0])

Делаем словарь с маппингом, чтобы можно было достать нужное значение по индексу. В значении -1 хранится None

In [60]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'CASE': Index(['nomn', 'gent', 'datv', 'accs', 'loct', 'ablt', 'loc2', 'voct', 'gen2'], dtype='object'),
 'DEPENDENCY': Index(['ROOT', 'obj', 'appos', 'parataxis', 'conj', 'advmod', 'ccomp', 'iobj',
        'csubj', 'case', 'obl', 'det', 'nsubj', 'cc', 'discourse', 'fixed',
        'mark', 'amod', 'advcl', 'xcomp', 'nummod', 'nmod', 'nsubj:pass',
        'nummod:gov', 'cop', 'acl:relcl', 'acl', 'aux', 'flat:foreign', 'punct',
        'obl:agent', 'flat', 'aux:pass', 'orphan', 'expl', 'csubj:pass'],
       dtype='object'),
 'GENDER': Index(['masc', 'femn', 'neut'], dtype='object'),
 'HEAD': Index(['обсуждаем', 'препарат', 'рисперидон', 'рисполепт', 'знаю', 'стоило',
        'меня', 'нет', 'проблем', 'лекарством',
        ...
        'привился', 'прививки', 'решать', 'неизвестно', 'быстро', 'проводились',
        'испытания', 'аукнется', 'вакцина', 'организме'],
       dtype='object', length=2514),
 'HEAD_ANIMACY': Index(['inan', 'ani

In [61]:

# labels_dict = {}
# test_labels_dict = {}

# for i in X.columns:
#   X[i], lbl = pd.factorize(X[i])
#   labels_dict[i] = lbl

# for i in X_test.columns:
#   X_test[i], lbl = pd.factorize(X_test[i])
#   test_labels_dict[i] = lbl 

# y = cor_df['COREFERENCE_CLUSTER']
# y, y_label = pd.factorize(y)

# y_test = cor_df_test['COREFERENCE_CLUSTER']
# y_test, y_test_label = pd.factorize(y_test)

In [161]:
df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 )]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,1,1,0.0,0.0,2.0,1,0,0,-1,1,2,1,0,0,0,1,1,2.0,1,0.799821,0.881828,0.914983
3,рисполепт,1,1,0.0,0.0,3.0,0,1,1,-1,1,3,2,1,0,0,1,1,3.0,0,0.813941,0.574603,0.914983
4,сперидан,1,1,0.0,0.0,4.0,-1,-1,0,-1,2,4,3,0,1,1,1,1,4.0,1,0.730241,0.680588,0.914983
21,лекарством,1,1,1.0,1.0,21.0,0,5,2,-1,1,4,7,-1,-1,-1,3,2,0.0,1,0.446905,0.552847,0.843088
142,рисполепта,1,1,1.0,13.0,142.0,0,0,1,-1,1,21,69,0,2,1,1,1,142.0,1,0.744327,0.509409,0.882947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14759,hifu,15,15,17.0,12.0,2167.0,-1,-1,-1,-1,-1,1,1903,-1,-1,-1,0,1,2167.0,-1,0.515243,0.504965,0.985836
14807,абраксан,15,15,18.0,3.0,2215.0,1,1,1,-1,1,1,2480,-1,-1,-1,0,1,2215.0,0,0.736828,0.413219,0.981547
14809,гемцитабином,15,15,18.0,3.0,2217.0,0,5,0,-1,1,21,2470,1,1,1,1,1,2217.0,1,0.599902,0.515348,0.981547
14856,фолфиринокс,15,15,18.0,8.0,2264.0,0,0,0,-1,1,21,1910,0,0,1,1,1,2264.0,1,0.679347,0.478782,0.981547


Разделим датасет на тестовый и тренировочный. for_train, for_test 

In [63]:
# train, test = train_test_split(df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)
# train, test = train_test_split(df, test_size=0.2)
train, test = train_test_split(df.loc[(df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)

In [64]:
train = train.reset_index()

In [65]:
test = test.reset_index()

In [66]:
test.head(5)

,index,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,7682,он,5,5,41.0,3.0,1484.0,-1,0,0,1,5,12,1427,-1,-1,0,0,3,1455.0,1,0.514413,0.575679,0.983559
1,11469,обезболивающие,9,9,0.0,34.0,300.0,-1,0,-1,-1,14,26,1557,0,2,1,1,2,301.0,0,0.590481,0.482550,0.979763
2,5889,препарат,4,4,6.0,6.0,336.0,0,0,0,-1,1,1,1158,-1,-1,-1,0,2,287.0,1,0.534916,0.492989,0.989138
3,9691,препараты,7,7,5.0,0.0,1152.0,0,0,0,-1,1,12,619,-1,-1,-1,13,2,0.0,0,0.457579,0.349272,0.972120
4,6254,препарата,5,5,3.0,1.0,56.0,0,1,0,-1,1,21,1264,0,1,2,1,2,41.0,1,0.462206,0.592557,0.992058


In [67]:
# test = test.iloc[:,:24].drop(['TOKEN', 'index'], axis=1)
# test = test.drop(['TOKEN', 'index'], axis=1)

In [68]:
# test.loc[test['NER'] != 0]

In [69]:
# ['TOPIC_NUM',	'POST_NUM',	'TOKEN_NUM',	'GENDER',	'PERSON', 'TOKEN', 'TOKEN_VECT', 'IS_ANSWER', 'SENT_NUM', 'HEAD_VECT', 'HEAD_CHILDS_VECT', 
#  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'CASE',	'POS',	'DEPENDENCY',	'HEAD',	'HEAD_POS']

In [70]:
# best set

# X, y, X_test, y_test = train.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',				'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
# train.COREFERENCE_CLUSTER, test.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',    'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',			'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

Делаем наборы для обучения моделей.

In [71]:
X, y, X_test, y_test = train.drop(['TOKEN','GENDER','NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
train.COREFERENCE_CLUSTER, test.drop(['TOKEN','GENDER', 'NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',    'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

In [72]:
# X, y, X_test, y_test = train.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], train.COREFERENCE_CLUSTER, test.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], test.COREFERENCE_CLUSTER

In [73]:
X.NER = [float(i) for i in X.NER]
X_test.NER = [float(i) for i in X_test.NER]


In [74]:
y

0      2058.0
1       449.0
2         0.0
3       658.0
4         0.0
        ...  
257       0.0
258     564.0
259      58.0
260     135.0
261     806.0
Name: COREFERENCE_CLUSTER, Length: 262, dtype: float64

Обучаем модели.

In [75]:
clf = tree.DecisionTreeClassifier(max_depth=2000, )
clf.fit(X, y)
clf.score(X_test, y_test)

0.7272727272727273

In [76]:
clf_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf_entropy.fit(X, y)
clf_entropy.score(X_test, y_test)

0.6666666666666666

In [77]:
randf_clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
randf_clf.fit(X, y)
randf_clf.score(X_test, y_test)

0.6818181818181818

In [78]:
regr = linear_model.LinearRegression()
regr.fit(X, y)
regr.score(X_test, y_test)


1.0

In [79]:
svm_clf = svm.SVC(decision_function_shape='ovo', cache_size=500)
svm_clf.fit(X, y)
svm_clf.score(X_test, y_test)

0.3787878787878788

In [80]:

knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
knn_clf.score(X_test, y_test)

0.696969696969697

In [81]:
# from sklearn.naive_bayes import MultinomialNB
# naive_clf = MultinomialNB().fit(X, y)


# **ОЦЕНКА МОДЕЛЕЙ**

In [82]:
def my_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y_test[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0 and test.iloc[:,22:23].iloc[[i]].values[0][0] in (2.0, 3.0):
      if X_predicted[i] == y_test[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')



In [83]:
pred = clf.predict(X_test)
pred_randf = randf_clf.predict(X_test)
pred_entropy = clf_entropy.predict(X_test)
pred_regr = regr.predict(X_test)
pred_svm = svm_clf.predict(X_test)
pred_knn = knn_clf.predict(X_test)
# naive_pred = naive_clf.predict(X_test)

In [84]:
# my_eval(list(naive_pred), list(y_test))

In [85]:
my_eval(list(pred_knn), list(y_test))

25  true positive
18  false positive
0  true negative
0  false negative
---------
0.5813953488372093  presicion
1.0  recall
0.7352941176470588  f-score


In [86]:
my_eval(list(pred_entropy), list(y_test))

22  true positive
22  false positive
0  true negative
0  false negative
---------
0.5  presicion
1.0  recall
0.6666666666666666  f-score


In [87]:
my_eval(list(pred), list(y_test))

26  true positive
18  false positive
0  true negative
0  false negative
---------
0.5909090909090909  presicion
1.0  recall
0.7428571428571429  f-score


In [88]:
my_eval(list(pred_randf), list(y_test))

23  true positive
21  false positive
0  true negative
0  false negative
---------
0.5227272727272727  presicion
1.0  recall
0.6865671641791045  f-score


In [89]:
my_eval(list(pred_regr), list(y_test))

7  true positive
59  false positive
0  true negative
0  false negative
---------
0.10606060606060606  presicion
1.0  recall
0.19178082191780824  f-score


In [90]:
my_eval(list(pred_svm), list(y_test))

3  true positive
18  false positive
0  true negative
0  false negative
---------
0.14285714285714285  presicion
1.0  recall
0.25  f-score


In [102]:
# df['TOPIC_NUM'] == 1
# df.loc[df['TOPIC_NUM'] == 1]

In [167]:
df.drop(['TOKEN','GENDER','NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', \
         'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'HEAD_ANIMACY',	'HEAD_CASE',	\
         'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18]

,TOKEN_NUM,NER,COREFERENCE_CLUSTER,TOKEN_VECT,SENT_VECT
0,0.0,0,0.0,0.478800,0.914983
1,1.0,0,0.0,0.534916,0.914983
2,2.0,1,2.0,0.799821,0.914983
3,3.0,1,3.0,0.813941,0.914983
4,4.0,1,4.0,0.730241,0.914983
...,...,...,...,...,...
14995,61.0,0,0.0,0.397897,0.981547
14996,62.0,0,0.0,0.506270,0.981547
14997,63.0,0,0.0,0.511537,0.981547
14998,64.0,0,0.0,0.444679,0.981547


In [179]:
list(df.values[1])[2:6] + list(df.values[1])[17:18] + list(df.values[1])[19:23]

[1,
 0.0,
 0.0,
 1.0,
 0,
 1,
 0.5349155068397522,
 0.5137307047843933,
 0.9149831533432007]

In [225]:
df.loc[(df['COREFERENCE_CLUSTER'] != 0) & (df['NER'] > 1)]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
366,препарата,1,1,2.0,5.0,366.0,0,1,0,-1,1,21,154,0,1,1,1,2,344.0,1,0.462206,0.440209,0.943423
421,препарат,1,1,5.0,0.0,421.0,0,0,0,-1,1,0,1,0,0,0,1,2,2.0,1,0.534916,0.881828,0.943423
429,препарат,1,1,6.0,0.0,429.0,0,0,0,-1,1,12,179,-1,-1,-1,0,2,2.0,1,0.534916,0.529394,0.943423
446,последнему,1,1,6.0,2.0,446.0,-1,2,0,-1,10,7,184,-1,-1,-1,0,2,442.0,1,0.554641,0.509320,0.943423
454,препарат,1,1,7.0,0.0,454.0,0,0,0,-1,1,12,189,-1,-1,-1,0,2,2.0,1,0.534916,0.488707,0.981541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14427,него,15,15,14.0,2.0,1835.0,-1,1,0,1,5,10,195,-1,-1,-1,9,3,1827.0,1,0.501803,0.571326,0.926079
14440,его,15,15,14.0,2.0,1848.0,-1,3,0,1,5,1,1175,-1,-1,-1,0,3,1827.0,1,0.480690,0.509213,0.926079
14478,него,15,15,14.0,6.0,1886.0,-1,1,0,1,5,21,2453,-1,-1,-1,15,3,1867.0,1,0.501803,0.491467,0.926079
14733,него,15,15,17.0,8.0,2141.0,-1,1,0,1,5,10,2278,-1,-1,1,0,3,2138.0,1,0.501803,0.574542,0.985836


# **ПОПАРНЫЙ МЕТОД**

In [242]:
topics = list(set(df.TOPIC_NUM.values))
topics


pair_anaphors = []
pair_antecedents = []
relation_list = []
counter = 0

for topic_num in topics:

  all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['NER'] > 1)]

  all_antecedents = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['NER'] == 1)]

  for anaphor in all_anaphors.values:
    # чтобы примеры влезли в память, и модель не переобучилась на антипримерах,
    # возьмем один положительный и два отрицательных класса для каждого анафора
    counter = 0
    for antecedent in all_antecedents.values:
      is_relation = 0

      # print(anaphor[-5], antecedent[5])
      if anaphor[-5] == antecedent[5]:
        is_relation = 1
        # такие срезы, чтобы убрать лишние данные, и в том числе разметку с ответами
        pair_anaphors.append(list(anaphor)[2:6] + list(anaphor)[17:18]\
                             + list(anaphor)[19:23])
        
        pair_antecedents.append(list(antecedent)[2:6] + list(antecedent)[17:18]\
                             + list(antecedent)[19:23])
        
        relation_list.append(is_relation)
      else:
        if counter < 3:

          pair_anaphors.append(list(anaphor)[2:6] + list(anaphor)[17:18]\
                              + list(anaphor)[19:23])
          
          pair_antecedents.append(list(antecedent)[2:6] + list(antecedent)[17:18]\
                              + list(antecedent)[19:23])
          
          relation_list.append(is_relation)
          counter += 1

  # counter += 1

print(pair_anaphors[0][0], pair_antecedents[0][0], relation_list[0])
len(pair_anaphors), len(pair_antecedents), len(relation_list)

1 1 0


(1160, 1160, 1160)

In [243]:
# можно посмотреть, что получилось
pair_anaphors[0]

[1,
 1.0,
 1.0,
 21.0,
 2,
 1,
 0.4469050467014313,
 0.5528465509414673,
 0.8430876135826111]

In [244]:
# проверим, третьим элементом должен быть номер токена
pair_anaphors[0][3]

21.0

In [245]:
# все сходится. мы взяли колонки с номерами поста, предложения и токена,\
# колонку NER, число, и вектора
df.loc[df.TOKEN_NUM == 21.0].iloc[[0]]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
21,лекарством,1,1,1.0,1.0,21.0,0,5,2,-1,1,4,7,-1,-1,-1,3,2,0.0,1,0.446905,0.552847,0.843088


In [246]:
cosine = [cosine_similarity([pair_anaphors[i]], [pair_antecedents[i]])[0][0] for i in range(len(pair_anaphors))]
# pair_df = pd.DataFrame(columns=['anaphs', 'ants', 'rel'])
pair_df = pd.DataFrame(columns=['cosine', 'rel'])

# pair_df['anaphs'] = pair_anaphors
# pair_df['ants'] = pair_antecedents
pair_df['cosine'] = cosine
pair_df['rel'] = relation_list

print(pair_df.loc[pair_df['cosine'] > 1])
print(pair_df.loc[pair_df['rel'] > 1])
pair_df

Empty DataFrame
Columns: [cosine, rel]
Index: []
Empty DataFrame
Columns: [cosine, rel]
Index: []


,cosine,rel
0,0.737681,0
1,0.887521,0
2,0.923855,0
3,0.661648,0
4,0.839553,0
...,...,...
1155,0.999999,1
1156,0.626041,0
1157,0.989071,0
1158,0.989644,0


In [247]:
pair_df.loc[pair_df.rel != 0]

,cosine,rel
12,0.999994,1
13,0.662112,1
17,0.661769,1
24,0.999997,1
25,0.661513,1
...,...,...
1113,0.999982,1
1123,0.999998,1
1142,0.999998,1
1155,0.999999,1


In [248]:
train, test = train_test_split(pair_df, test_size=0.2)
train = train.reset_index()
test = test.reset_index()

In [249]:
X, y, X_test, y_test = train.iloc[:, 1:2], train.rel, test.iloc[:, 1:2], test.rel


In [250]:
train.iloc[:1, 1:2]

,cosine
0,0.975927


In [251]:
test.iloc[:1, 1:2]

,cosine
0,0.984462


In [252]:
regr = linear_model.LinearRegression()
regr.fit(X, y)

print(regr.score(X_test, y_test))
print('-----')
pred = regr.predict(X_test)
rule_eval(pred, y_test)

0.1754822805147904
-----
0  true positive
232  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [253]:
knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1)
knn_clf.fit(X, y)

print(knn_clf.score(X_test, y_test))
print('-----')
pred = knn_clf.predict(X_test)
rule_eval(pred, y_test)

0.9482758620689655
-----
29  true positive
5  false positive
191  true negative
7  false negative
---------
0.8529411764705882  presicion
0.8055555555555556  recall
0.8285714285714286  f-score


In [254]:
dtclf = tree.DecisionTreeClassifier(max_depth=2000, )
dtclf.fit(X, y)

print(dtclf.score(X_test, y_test))
print('-----')
pred = dtclf.predict(X_test)
rule_eval(pred, y_test)


0.9396551724137931
-----
29  true positive
7  false positive
189  true negative
7  false negative
---------
0.8055555555555556  presicion
0.8055555555555556  recall
0.8055555555555556  f-score


In [255]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf.fit(X, y)

print(clf.score(X_test, y_test))
print('-----')
pred = clf.predict(X_test)
rule_eval(pred, y_test)

0.9396551724137931
-----
29  true positive
7  false positive
189  true negative
7  false negative
---------
0.8055555555555556  presicion
0.8055555555555556  recall
0.8055555555555556  f-score


In [256]:
clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
clf.fit(X, y)

print(clf.score(X_test, y_test))
print('-----')
pred = clf.predict(X_test)
rule_eval(pred, y_test)

0.9396551724137931
-----
29  true positive
7  false positive
189  true negative
7  false negative
---------
0.8055555555555556  presicion
0.8055555555555556  recall
0.8055555555555556  f-score


In [ ]:
# knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
# knn_clf.score(X_test, y_test)

In [ ]:
# clf = tree.DecisionTreeClassifier(max_depth=2000, )
# clf.fit(X, y)
# clf.score(X_test, y_test)